## Using Abeta pattern

In [2]:
import nibabel as nib
from nilearn.image import resample_to_img, smooth_img, resample_img, new_img_like
import numpy as np
from nilearn.masking import apply_mask, unmask
from nilearn.datasets import load_mni152_brain_mask, load_mni152_brain_mask

mni152 = nib.load('./example_img.nii')
brain_wo_voi_mask = load_mni152_brain_mask()
brain_2mm_mask = resample_to_img(brain_wo_voi_mask, mni152, interpolation='nearest')

def preprocess_img(pet_img, downsample=3):
    pet_img = smooth_img(pet_img, fwhm=8)
    data = pet_img.get_fdata()
    data[np.isnan(data)] = 0
    pet_img = nib.nifti1.Nifti1Image(data, pet_img.affine, pet_img.header)
    downsample_affine = pet_img.affine.copy()
    downsample_affine[:3, :3] *= downsample
    pet_img = resample_img(pet_img, target_affine=downsample_affine)
    return pet_img

def scale_intensity(X:np.array):
    return (X - X.min(axis=1).reshape((-1, 1))) / (X.max(axis=1) - X.min(axis=1)).reshape(-1,1)

def flip_image(img):
    data = img.get_fdata()
    data = np.flip(data, 0)
    new_image = new_img_like(img, data)
    return new_image

def whole_brain_to_half(img, mask):
    return [apply_mask(flip_image(img), mask), apply_mask(img, mask)]

def half_to_whole_brain(half_imgs, mask):
    return unmask(half_imgs[0], mask)+unmask(half_imgs[1], mask)

In [ ]:
# if you wish to build a logistic regression model

# from sklearn.linear_model import LogisticRegression
# import numpy as np
# from sklearn.metrics import roc_auc_score, roc_curve, auc

# X = []
# for img in images:
#     img = preprocess_img(img, downsample=3)
#     X.extend(whole_brain_to_half(img, brain_2mm_mask))
# X = scale_intensity(np.array(X))
# y = ...

# lr = LogisticRegression(max_iter=50000, class_weight='balanced')
# lr.fit(X, y)
# print(roc_auc_score(y_test, lr.decision_function(X_test)))
# print(classification_report(y_test, lr.predict(X_test)))

In [4]:
# to use the trained model
# please note that pet images should be normalized first!

from pathlib import Path

import pickle
with open('./LCAD_LCCN_lr_model.pkl', 'rb') as f:
    lr = pickle.load(f)


downsample_ratio = 3
example_img = nib.load('./example_img.nii')
example_img = preprocess_img(example_img, downsample_ratio)
resampled_mask = resample_to_img(brain_2mm_mask, example_img, interpolation='nearest')
example_img = apply_mask(example_img, resampled_mask)
X = scale_intensity(np.array([example_img]))
lr.decision_function(X), lr.predict(X)

(array([6.10816421]), array([1]))

## Using DCCC to calculate Centiloid/CenTauR

In [4]:
from pathlib import Path
import os
import nibabel as nib
import torch

from evaluator import CentiloidCalculator, CenTauRCalculator
from nets import AffineVoxelMorphONNX, RegressorModel
from utils import DeepCascadeSpatialNormalizer
from nets import RegressorModel, AffineVoxelMorphONNX
from constants import RIGID_INSHAPE
mask = nib.load("./templates/padded_mask.nii")
mask = (
    torch.from_numpy(mask.get_fdata()).reshape((1, 1, *mask.shape))
)
rigid_model = RegressorModel(RIGID_INSHAPE)
rigid_model.load_state_dict(torch.load('./data_and_models/best_RegressorModel.pth'))
rigid_model.eval()
affine_model = AffineVoxelMorphONNX(mask)
affine_model.load_state_dict(torch.load('./data_and_models/best_AffineVoxelMorph.pth'))
affine_model.eval()

cl_calc = CentiloidCalculator()

normalizer = DeepCascadeSpatialNormalizer(rigid_model, affine_model, 1)
normalizer.normalize(
    [
        Path('./data_and_models/ED25_AV45.nii'),
        Path('./data_and_models/YC12_AV45.nii')
    ],
    [
        Path('./warped1.nii'),
        Path('./warped2.nii')
    ])
# you can calibrate our calculator based on our paper, but actually the difference is very small
print(cl_calc.calculate('./warped1.nii', 'av45')*1.05-6.98) # 378784, GT: 59.7
print(cl_calc.calculate('./warped2.nii', 'av45')*1.05-6.98) # 462623, GT: -2.8
os.remove('./warped1.nii')
os.remove('./warped2.nii')

e:\programs\Anaconda\envs\dlmed\lib\site-packages\monai\networks\blocks\warp.py:72: UserWarning: monai.networks.blocks.Warp: Using PyTorch native grid_sample.
  warnings.warn("monai.networks.blocks.Warp: Using PyTorch native grid_sample.")


58.71800503373146
-3.142954159975064
